In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [3]:
import spacy
from spacy.tokens import DocBin
from datasets import load_dataset
import transformers

In [ ]:
dataset = load_dataset("conll2003")
nlp = spacy.blank("en")
doc_bin = DocBin()

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [6]:
train_examples = dataset["train"]['tokens']
train_tags = dataset["train"]['ner_tags']

In [7]:
print(train_examples[0])
print(train_tags[0])

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]


In [8]:
labels = dataset["train"].features["ner_tags"].feature.names

In [9]:
for tokens, ner_tags in zip(dataset["train"]["tokens"], dataset["train"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

In [10]:
for tokens, ner_tags in zip(dataset["validation"]["tokens"], dataset["validation"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("dev.spacy")

In [11]:
!python -m spacy init config config.cfg --lang en --pipeline ner

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy --training.max_epochs 10 --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     44.28    0.00    0.00    0.00    0.00
  0     200        156.78   3000.96   50.66   52.03   49.36    0.51
  0     400        507.47   2122.70   69.54   72.37   66.91    0.70
  0     600        347.62   1974.91   76.62   77.75   75.52    0.77
  0     800        348.17   2083.60   82.65   84.95   80.47    0.83
  0    1000        435.35   2261.76   85.81   85.69   85.94    0.86
  1    1200        467.04   2165.55   88.77   89.39   88.17    0.89
  1    1400        505.94   1907.21   89.55   89.04   90.07    0.90
  1    1600        651.82   

In [16]:
for tokens, ner_tags in zip(dataset["test"]["tokens"], dataset["test"]["ner_tags"]):
    doc = nlp.make_doc(" ".join(tokens))
    ents = []
    start = 0
    for i, (word, tag_id) in enumerate(zip(tokens, ner_tags)):
        tag = labels[tag_id]
        if tag == "O":
            start += len(word) + 1
            continue
        if tag.startswith("B-"):
            ent_start = start
            ent_end = start + len(word)
            ent_label = tag[2:]

            for j in range(i+1, len(tokens)):
                next_tag = labels[ner_tags[j]]
                if next_tag == f"I-{ent_label}":
                    ent_end += len(tokens[j]) + 1
                else:
                    break
            ents.append((ent_start, ent_end - 1, ent_label))
        start += len(word) + 1

    doc.ents = [doc.char_span(start, end + 1, label=label) for start, end, label in ents if doc.char_span(start, end + 1, label=label)]
    doc_bin.add(doc)

doc_bin.to_disk("test.spacy")

In [17]:
!python -m spacy evaluate output/model-best ./test.spacy --output metrics.json

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   92.38 
NER R   92.28 
NER F   92.33 
SPEED   25971 


=============================== NER (per type) ===============================

           P       R       F
ORG    88.22   90.98   89.58
MISC   91.94   90.27   91.10
PER    94.02   92.11   93.06
LOC    94.89   94.53   94.71

✔ Saved results to metrics.json
